In [ ]:
import pandas
import theano.tensor as T
import numpy as np
import pymc3 as pm
from functools import reduce
from preprocessing import save_sequences, extract_amounts_on_board, get_protocols
import matplotlib.pyplot as plt
import math
import datetime
from itertools import chain
import pickle

fullCsvName = './data/alldata.csv'
protocolCsvName = './data/protocol.csv'
binSize = 0.5
segmentDays = 30
startDate = '2016-11-01'
endDate   = '2017-11-01'

startDates = []
endDates = []
while startDate < endDate:
    startDates.append(startDate)
    startDate = datetime.datetime.strftime(
                    datetime.datetime.strptime(startDate, "%Y-%m-%d") + datetime.timedelta(segmentDays), 
                    "%Y-%m-%d"
                )
    endDates.append(startDate)
    
# Example
get_protocols(protocolCsvName, '2016-11-01', '2017-02-01')

In [ ]:
allData = []
for (iSegment, startDate) in enumerate(startDates):

    # Get data about what humans inferred parameters to be (not used for inference, just comparison)
    segmentProtocols = get_protocols(protocolCsvName, startDate, endDates[iSegment])
    humanCFs = list(set(chain(*[segmentProtocols[label].tolist() 
                      for label in ["CorrFactDay", "CorrFactNight"]])))
    humanCRs = list(set(chain(*[segmentProtocols[label].tolist() 
                      for label in ["CarbRatioBreakfast", "CarbRatioBreakfast", "CarbRatioSnack", "CarbRatioDinner", "CarbRatioBedtime"]])))  
    humanBasals = list(set(segmentProtocols["Lantus"].tolist()))
    
    # Get data about what we actually did
    segmentCsvName = './processed/segment_start_' + str(startDate) + '.csv'
    save_sequences(fullCsvName, segmentCsvName, b=binSize, d=3, segmentDays=segmentDays, segmentStart=startDate, 
                   sequenceLength=15, maxGapHours=16)
    df = pandas.read_csv(segmentCsvName)

    (basal, smbg, newCarbs, newInsulin, delT, insActCurve, carbActCurve,
            currentIOB, currentCOB, iobImpact, cobImpact, nSeq, nSeg) = extract_amounts_on_board(df, binSize)

    with pm.Model() as model:

        M = pm.Uniform('Basal', lower=-40, upper=40) # Prior on basal metabolism
        CF = pm.Normal('CF', mu=250, sd=100) # Prior on correction factor
        CR = pm.Normal('CR', mu=40, sd=25) # Prior on correction factor
        sigmaBG = pm.Normal('SigmaBG', mu=50, sd=10) # Prior on measurement error, assuming
            # normally distributed. Note that log-normal distribution of glucose measurement 
            # would be more realistic, but a lot slower to run.
        # tauBG = 0.095; # SD of log(BG). If 95% obs w/i 20%, then SD ~ 10%. log(1.1) = 0.095.

        glus = []
        glus.append(pm.Normal('glucose0', mu=150, sd=80, observed=smbg[0]))

        # TODO: infer true IOB, COB

        for i in range(1, nSeg):
            print(i)
            # Calculate predicted next glucose value, based on...
            glus.append(pm.Normal('glucose' + str(i), 
                                  mu = glus[i-1] + # Last true glucose value
                                       -1 * np.dot(newInsulin[i-1], insActCurve) * CF + # Newly-added insulin
                                       -1 * np.transpose([np.dot(currentIOB[i-1][j], iobImpact[i][j]) for j in range(nSeq)]) * CF +   # All insulin, including new
                                       np.dot(newCarbs[i-1], carbActCurve) * CF / CR + # Newly-added carbs
                                       np.transpose([np.dot(currentCOB[i-1][j], cobImpact[i][j]) for j in range(nSeq)]) * CF / CR + # Carbs on board
                                       (M - basal[i-1]) * delT[i-1]/24 * CF, # Basal/metabolism mismatch
                                  sd = sigmaBG, 
                                  observed=smbg[i]))

        # Inference!
        trace = pm.sample(4000, tune=1000, progressbar=True) # draw posterior samples using NUTS sampling

    #plt.figure(figsize=(7, 7))
    #pm.traceplot(trace[100:])
    #plt.tight_layout();

    stats = pm.summary(trace)
    print('Segment beginning ', startDate)
    print(stats)
    allData.append({
        'stats': stats,
        'trace': trace,
        'humanCFs': humanCFs,
        'humanCRs': humanCRs,
        'humanBasals': humanBasals,
        'startDate': startDate,
        'endDate': endDates[iSegment]
    })
    

    with open('results.pickle', 'wb') as handle:
        pickle.dump(allData, handle)
    
    print('backed up to results.pickle')

In [ ]:
allDataBackup = allData

with open('results.pickle', 'rb') as handle:
    allData = pickle.load(handle)
    
for iSegment in range(len(allData)):
    allData[iSegment]['startDate'] = startDates[iSegment]

In [ ]:
fig, axs = plt.subplots(nrows=3, ncols=1, sharex=True, figsize=(11,11))
plt.xticks(rotation=270)
axs[0].set_title('Correction factors')
axs[0].set_ylabel('mg/dL / unit')
axs[1].set_title('Carb ratios')
axs[1].set_ylabel('g / unit')
axs[2].set_title('Basal dose')
axs[2].set_ylabel('units')
    
for iSegment in range(10): #len(allData)):
    data = allData[iSegment]
    axs[0].plot([data['startDate']] * len(data['humanCFs']), data['humanCFs'], 'ro')
    axs[1].plot([data['startDate']] * len(data['humanCRs']), data['humanCRs'], 'ro')
    hHuman = axs[2].plot([data['startDate']] * len(data['humanBasals']), data['humanBasals'], 'ro')
    
    inferred = {
        label: data['stats'].at[label, 'mean'] for label in ['CF', 'CR', 'Basal']
    }
    inferredError = {
        label: [inferred[label] - data['stats'].at[label, 'hpd_2.5'],
                data['stats'].at[label, 'hpd_97.5'] - inferred[label]] for label in ['CF', 'CR', 'Basal']
    }
    
    axs[0].errorbar([data['startDate']], [inferred['CF']], yerr=[inferredError['CF']],    fmt='bo')
    axs[1].errorbar([data['startDate']], [inferred['CR']],    yerr=[inferredError['CR']],    fmt='bo')
    hInferred = axs[2].errorbar([data['startDate']], [inferred['Basal']], yerr=[inferredError['Basal']], fmt='bo')

fig.legend((hHuman[0], hInferred), ('Protocol sheet', 'Inferred value'), loc=1)

fig.savefig('./processed/human_vs_inferred.png')